In [1]:
!pip install langgraph langchain_groq langchain langchain-community faiss-cpu pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successf

In [2]:
#Importing libraries
import os
from typing import TypedDict, Dict
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables.graph import MermaidDrawMethod
from langchain_groq import ChatGroq
from IPython.display import display, Image

In [3]:
#Creating the state class
class State(TypedDict):
    query: str
    category: str
    sentiment: str
    response: str

In [1]:
from google.colab import userdata
groq_api_key=userdata.get('groq')

In [5]:
#initialising the llm
llm=ChatGroq(
   groq_api_key=userdata.get('groq'),
   model_name="llama-3.3-70b-versatile",
   temperature=0.7

)

In [6]:
# Generic helper to run a prompt through the LLM
def run_prompt(template: str, query: str) -> str:
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | llm
    return chain.invoke({"query": query}).content

# Categorizing function
def categorize(state: State) -> State:
    template = (
        "Categorize the following customer query into one of these categories: "
        "Technical, Billing, General. Query: {query}"
    )
    return {"category": run_prompt(template, state["query"])}

# Sentiment analysis function
def analyze_sentiment(state: State) -> State:
    template = (
        "Analyze the sentiment of the following customer query. "
        "Respond with either 'Positive', 'Neutral', or 'Negative'. Query: {query}"
    )
    return {"sentiment": run_prompt(template, state["query"])}

# Handlers for different categories
def handle_technical(state: State) -> State:
    return {
        "response": run_prompt(
            "Provide a technical support response to the following query: {query}",
            state["query"]
        )
    }

def handle_billing(state: State) -> State:
    return {
        "response": run_prompt(
            "Provide a billing support response to the following query: {query}",
            state["query"]
        )
    }

def handle_general(state: State) -> State:
    return {
        "response": run_prompt(
            "Provide a general support response to the following query: {query}",
            state["query"]
        )
    }

# Escalate function
def escalate(state: State) -> State:
    return {
        "response": (
            "This query has been escalated to a human agent due to its negative sentiment."
        )
    }


In [7]:
# handling the routing function
def route_query(state: State) -> State:
    if state['sentiment'] == 'Negative':
        return "escalate"
    elif state['category'] == 'Technical':
        return "handle_technical"
    elif state['category'] == 'Billing':
        return "handle_billing"
    else:
        return "handle_general"

In [8]:
#crafting the workflow
workflow = StateGraph(State)
workflow.add_node("categorize", categorize)
workflow.add_node("analyze_sentiment", analyze_sentiment)
workflow.add_node("handle_technical", handle_technical)
workflow.add_node("handle_billing", handle_billing)
workflow.add_node("handle_general", handle_general)
workflow.add_node("escalate", escalate)

#adding edges
workflow.add_edge("categorize", "analyze_sentiment")
workflow.add_conditional_edges("analyze_sentiment",
route_query,{
    "handle_technical" : "handle_technical",
    "handle_billing": "handle_billing",
    "handle_general" : "handle_general",
    "escalate": "escalate"
})
workflow.add_edge("handle_technical", END)
workflow.add_edge("handle_billing", END)
workflow.add_edge("handle_general", END)
workflow.add_edge("escalate", END)

#aadding the entry point
workflow.set_entry_point("categorize")

#compile the workflow
app = workflow.compile()

In [9]:
#Function to run customer support
def run_customer_support(query: str) -> dict:
    result = app.invoke({"query": query})
    return {
        "category": result['category'],
        "sentiment": result['sentiment'],
        "response": result['response'],
    }

In [10]:
#testing the output
query = "My internet connection is gone it's not working, Can you help me?"
result = run_customer_support(query)
print(f"Query: {query}")
print(f"Category: {result['category']}")
print(f"Sentiment: {result['sentiment']}")
print(f"Response: {result['response']}")
print("\n")

Query: My internet connection is gone it's not working, Can you help me?
Category: I would categorize the customer query as: **Technical**

The customer is reporting a problem with their internet connection, which suggests a technical issue that needs to be resolved. They are seeking assistance to troubleshoot or fix the problem, which is a classic example of a technical support query.
Sentiment: Negative.
Response: I'm so sorry to hear that your internet connection is not working. I'd be happy to help you troubleshoot the issue.

Can you please tell me a bit more about what's happening? For example:

* Are you getting any error messages?
* Have you tried restarting your router or modem?
* Are all of your devices (e.g. laptop, phone, tablet) affected, or just one?
* Have you made any recent changes to your internet setup or equipment?

Additionally, you can try a few basic troubleshooting steps to see if they resolve the issue:

* Restart your router and modem by unplugging them, waiti

In [11]:
# Install Gradio if not already installed
!pip install gradio


In [12]:
import gradio as gr
def simple_chatbot(message, history):
    result = run_customer_support(message)
    return result['response']

demo = gr.ChatInterface(
    fn=simple_chatbot,
    title="Customer Support Chatbot",
    description="Ask your customer support questions here",
    examples=["My internet isn't working", "Billing question", "General inquiry"]
)
demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:348: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://83cbe55d1259ee0e24.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
